In [1]:
from pathlib import Path

import pandas as pd

In [2]:
dates_to_visualize = [
                                                                                            "2018-09", "2018-10", "2018-11", "2018-12",
    "2019-01", "2019-02", "2019-03", "2019-04", "2019-05", "2019-06", "2019-07", "2019-08", "2019-09", "2019-10", "2019-11", "2019-12", 
    "2020-01", "2020-02", "2020-03", "2020-04", "2020-05", "2020-06", "2020-07", "2020-08", "2020-09", "2020-10", "2020-11", "2020-12",
    "2021-01", "2021-02", "2021-03", "2021-04", "2021-05", "2021-06", "2021-07", "2021-08", "2021-09", "2021-10", "2021-11", "2021-12",
    "2022-01", "2022-02", "2022-03", "2022-04", "2022-05", "2022-06", "2022-07", "2022-08", "2022-09", "2022-10", "2022-11", "2022-12",
    "2023-01", "2023-02",
    # "2024-02",
]

In [3]:
available_data_files = list(Path("../data/httparchive_metrics/nel_domain_resource_monitoring_stats").glob("*.parquet"))

used_data_files = list(filter(lambda file: file.stem in dates_to_visualize, available_data_files))
used_data_files

[WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2018-09.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2018-10.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2018-11.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2018-12.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2019-02.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2019-03.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2019-04.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2019-05.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2019-06.parquet'),
 WindowsPath('../data/httparchive_metrics/nel_domain_resource_monitoring_stats/2019-07.parquet'),
 WindowsPath('../dat

### Aggregate result to visualize

In [4]:
from results.result_utils import date_to_text_format, concat_data_from_files

result = concat_data_from_files(used_data_files)

result['date_formatted'] = result['date'].map(date_to_text_format)

result

,date,url_domain,url_domain_hosted_resources,url_domain_hosted_resources_with_nel,url_domain_monitored_resources_ratio,date_formatted
0,2018-09,m2.cn.bing.com,159,1,0.630000,Sep 2018
1,2018-09,www4.bing.com,179,1,0.560000,Sep 2018
0,2018-10,classifieds.seloc.org,124,68,54.840000,Oct 2018
1,2018-10,cpqa.catchpoint.com,1,1,100.000000,Oct 2018
2,2018-10,forums.seloc.org,10,1,10.000000,Oct 2018
...,...,...,...,...,...,...
2258974,2023-02,zzzw.de,63,62,98.410004,Feb 2023
2258975,2023-02,zzzz.bg,6,6,100.000000,Feb 2023
2258976,2023-02,zzzz.tw,42,42,100.000000,Feb 2023
2258977,2023-02,zzzzap.nl,37,37,100.000000,Feb 2023


### NEL Domain monitored resources ratio distribution

In [5]:
from results.result_utils import get_first_or_0

distribution_result = pd.DataFrame({}, index=[
    '(0%-10%>',
    '(10%-25%>',
    '(25%-50%>',
    '(50%-75%>',
    '(75%-100%)',
    '<100%>',
])

for month in dates_to_visualize:
    current_month_data = result[result['date'] == month].copy()
    
    month_data_col = []

    # month_result['(0%-10%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 0.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 10.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(10%-25%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 10.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 25.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(25%-50%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 25.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 50.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(50%-75%>']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 50.0) & (current_month_data['url_domain_monitored_resources_ratio'] <= 75.0)].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['(75%-100%)']    
    next_val = current_month_data[(current_month_data['url_domain_monitored_resources_ratio'] > 75.0) & (current_month_data['url_domain_monitored_resources_ratio'] < 100.0)].count()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['<100%>']
    next_val = current_month_data[current_month_data['url_domain_monitored_resources_ratio'] == 100.0].count()
    month_data_col.append(get_first_or_0(next_val))
    
    
    distribution_result[date_to_text_format(month)] = month_data_col
    
distribution_result

,Sep 2018,Oct 2018,Nov 2018,Dec 2018,Jan 2019,Feb 2019,Mar 2019,Apr 2019,May 2019,Jun 2019,...,May 2022,Jun 2022,Jul 2022,Aug 2022,Sep 2022,Oct 2022,Nov 2022,Dec 2022,Jan 2023,Feb 2023
(0%-10%>,2,5,10,18,0,29,26,134,186,141,...,0,0,22446,26863,30456,22709,24243,25498,25761,24930
(10%-25%>,0,2,5,7,0,7,15,197,127,215,...,0,0,7062,4148,4315,2686,2840,2840,2661,2444
(25%-50%>,0,1,2,5,0,8,12,3593,2371,3520,...,0,0,5421,26613,7469,7870,7435,6964,6759,5978
(50%-75%>,0,1,3,3,0,25,29,26203,19313,26300,...,0,0,4702,60435,7145,10416,9453,9035,8380,7981
(75%-100%),0,0,6,9,0,29,38,8478,9771,9362,...,0,0,75909,150725,102790,109367,108127,108117,102015,106233
<100%>,0,1,161,333,0,271,262,35983,49118,39675,...,0,0,1117469,1815855,1897987,2025750,2151810,2162282,2147097,2111383
